# Homework 3: Intoduction to Data Science

## Due Tuesday, June 5, start of class

### Jake Pitkin

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import scipy as sc
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import time

### 1. Scrape Github Repository List and Repository Information using BeautifulSoup

### Task 1.2 Download the website

In [49]:
# Download the first 10 pages of the highly starred git repos
for i in range(1, 11):
    url = "https://github.com/search?o=desc&p=" + str(i) + "&q=stars%3A%3E1&s=stars&type=Repositories"
    # here we actually access the website
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')

    # save the file
    with open("git_pages/page_" + str(i) + ".html", 'w') as new_file:
        new_file.write(html)
    time.sleep(10)

# Read 10 pages from disk and create soup objects
pages = []
for i in range(1, 11):
    page = BeautifulSoup(open("git_pages/page_" + str(i) + ".html"), "html.parser")
    pages.append(page)
    
# Extract the repository links
repo_links = []
for i in range(10):
    for repo_link in pages[i].select(".v-align-middle"):
        try:
            repo_links.append("https://github.com/" + repo_link['href'])
        except:
            pass

# Download each repository page
for i, repo_link in enumerate(repo_links):
    # here we actually access the website
    with urllib.request.urlopen(repo_link) as response:
        html = response.read()
        html = html.decode('utf-8')

    # save the file
    with open("repo_pages/repo_" + str(i) + ".html", 'w') as new_file:
        new_file.write(html)
    time.sleep(10) 

### 1.3. Extract Data

In [106]:
# Extract the repository links from saved html files
pages = []
for i in range(1, 11):
    page = BeautifulSoup(open("git_pages/page_" + str(i) + ".html"), "html.parser")
    pages.append(page)
repo_links = []
for i in range(10):
    for repo_link in pages[i].select(".v-align-middle"):
        try:
            repo_links.append("https://github.com/" + repo_link['href'])
        except:
            pass

# Read repo pages from disk and create soup objects
repo_pages = []
for i in range(100):
    page = BeautifulSoup(open("repo_pages/repo_" + str(i) + ".html"), "html.parser")
    repo_pages.append(page)

# Create an entry for each repository
project_info = pd.DataFrame(columns=['name', 'url', 'commits', 'forks', 'contributors', 'stars', 'issues', 'len_readme'])

for index, repo_page in enumerate(repo_pages):
    try:
        url = repo_links[index]
        name = url.split('/')[-1]
        commits = repo_page.select(".commits")[0].select(".num")[0].get_text().replace(" ", "").replace("\n", "")
        forks = repo_page.select(".social-count")[2].get_text().replace(" ", "").replace("\n", "")
        contributors = repo_page.select(".numbers-summary")[0].select(".num")[3].get_text().replace(" ", "").replace("\n", "")
        stars = repo_page.select(".social-count")[1].get_text().replace(" ", "").replace("\n", "")
        issues = repo_page.select(".Counter")[0].get_text().replace(" ", "").replace("\n", "")
        len_readme = len(repo_page.select("#readme")[0].get_text())
    except:
        print("fail")
        continue
    project_info = project_info.append({'name':name, 'url':url, 'commits':commits, 'forks':forks,
                                        'contributors':contributors, 'stars':stars, 'issues':issues, 
                                        'len_readme':len_readme}, ignore_index=True)

# Save dataframe to disk
project_info.to_csv("project_info.csv", sep=',')

### 2. Use linear regression to analyze the Github repository data

In [ ]:
# Load in the csv
project_info = pd.read_csv("project_info.csv", sep=',')